In [ ]:
# Set the environment variable in a terminal
export ACLED_API_KEY="your-api-key-here"
export ACLED_EMAIL="you@youremail.com"

In [ ]:
import os

# Get the API key and email from the environment variables
api_key = os.environ.get("ACLED_API_KEY")
email = os.environ.get("ACLED_EMAIL")

In [ ]:
import requests
import pandas as pd

api_key = "your_api_key"
email = "your_email"
url_base = "https://api.acleddata.com/acled/read/"

# Set the initial last event date and offset
last_event_date = "2023-02-01"
offset = 0

# Loop until there is no more data to retrieve
while True:
    # Define the API endpoint URL with the API key and email as query parameters
    url = f"{url_base}?key={api_key}&email={email}&country=Ukraine&event_date=2022-02-01|{last_event_date}&event_date_where=BETWEEN&limit=500&offset={offset}"

    # Make the HTTP GET request
    response = requests.get(url)

    # Check if the request was successful (200 status code)
    if response.status_code == 200:
        # Process the data
        data = response.content

        # Save the response as a CSV file
        with open(f"acled_data_{offset}.csv", "wb") as f:
            f.write(data)

        # Convert the CSV data to a DataFrame
        df = pd.read_csv(f"acled_data_{offset}.csv", encoding="utf-8-sig")

        # Concatenate the DataFrame with the previous data
        if offset == 0:
            all_data = df
        else:
            all_data = pd.concat([all_data, df])

        # Check if there is more data to retrieve
        if len(df) < 500:
            break
        else:
            # Update the last event date and offset to retrieve the next set of data
            last_event_date = df.iloc[-1]["event_date"]
            offset += 500

# Save all the data to a single CSV file
all_data.to_csv("acled_data.csv", index=False, encoding="utf-8-sig")
